## musdb, museval pip을 이용해 설치 필요

In [69]:
import musdb
import museval
import numpy as np


## 곡 DB 로드

In [67]:
mus = musdb.DB(root_dir='/home/scpark/hard/datasets/musdb18')

# load the training tracks
tracks = mus.load_mus_tracks(subsets=['test'])

## 한 곡을 평가하는 방법, museval.eval_mus_track

In [51]:
'''
평가하고 싶은 항목을 track.targets[''].audio와 같은 shape으로 만들어 estimates dict에 담는다.
2가지 시나리오가 존재
1. vocals, drums, bass, other를 넣어 평가
2. vocals, accompaniment를 넣어 평가
예제는 true 값을 넣음
'''
estimates = {
    'vocals': track.targets['vocals'].audio,
    'drums': track.targets['drums'].audio,
    'bass': track.targets['bass'].audio,
    'other': track.targets['other'].audio,
    #'accompaniment': track.targets['accompaniment'].audio
}

'''
평가하고 score 출력 (꽤 시간 오래 걸림)
output_dir 디렉토리에 한 곡 내에서 구간별로 평가 점수가 저장 됨.
리턴 하는 scores는 구간별 점수들의 mode값
'''
scores = museval.eval_mus_track(track, estimates, output_dir=None)
print(scores)


vocals              => SDR:nandB, SIR:201.211dB, ISR:206.147dB, SAR:200.985dB, 
drums               => SDR:nandB, SIR:199.483dB, ISR:216.903dB, SAR:199.423dB, 
bass                => SDR:nandB, SIR:163.810dB, ISR:181.886dB, SAR:163.716dB, 
other               => SDR:nandB, SIR:167.656dB, ISR:186.850dB, SAR:167.641dB, 



/home/scpark/anaconda3/envs/ai/lib/python3.6/site-packages/museval/__init__.py:105: RuntimeWarning: Mean of empty slice
  ) + "dB, "


## 짧은 구간을 평가하는 방법, museval.evaluate  

In [66]:
# 예를 들기 위해 원본에서 앞에서부터 3초씩 잘라서 넣는다.
slices = slice(0, 3 * 44100)

references = [track.targets['vocals'].audio[slices], 
            track.targets['drums'].audio[slices],
            track.targets['bass'].audio[slices],
            track.targets['other'].audio[slices]]

# shape=(nsources, nsamples, nchannels)
references = np.stack(references)

estimates = [track.targets['vocals'].audio[slices], 
            track.targets['drums'].audio[slices],
            track.targets['bass'].audio[slices],
            track.targets['other'].audio[slices]]

# references와 같은 shape을 가져야 한다.
# shape=(nsources, nsamples, nchannels)
estimates = np.stack(estimates)

# 1초씩 잘라서 비교하면서 평가
# 각각 (nsources, nframes) shape을 갖는다. 
# vocals, drums, bass, other 3초짜리를 넣고, win=1*44100, hop=1*44100로 두면 shape이 (4, 3)으로 나온다.
SDR, ISR, SIR, SAR = museval.evaluate(references, estimates,
                                     win=1*44100, hop=1*44100)

print("SDR")
print(SDR)

print("ISR")
print(ISR)

print("SIR")
print(SIR)

print("SAR")
print(SAR)

SDR
[[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]]
ISR
[[204.94067199 206.19728497 203.82422185]
 [239.10249602 238.04221711 240.16363901]
 [173.80268686 212.02936332 212.73441786]
 [209.29521613 202.85421671 203.5850656 ]]
SIR
[[200.53572868 201.73617937 198.81060343]
 [196.88387982 194.6622243  200.86987338]
 [133.86242346 170.65393665 175.35296853]
 [193.83825493 182.74350204 182.34079141]]
SAR
[[200.93216635 202.1282304  199.15195735]
 [196.88184177 194.66086032 200.86843079]
 [133.86287555 170.65438666 175.35463944]
 [193.79154905 182.7676587  182.37257482]]
